In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import time
import re

import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook_connected"


In [4]:
DATABASE_NAME = "internet_governance_news.db"

def create_database():
    conn = sqlite3.connect(DATABASE_NAME)
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS articles (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT,
            date TEXT,
            author TEXT,
            url TEXT UNIQUE,
            source TEXT
        )
    """)
    conn.commit()
    conn.close()
    print("✅ Banco e tabela 'articles' prontos!")

create_database()


✅ Banco e tabela 'articles' prontos!


In [5]:
def insert_article(title, date, author, url, source):
    conn = sqlite3.connect(DATABASE_NAME)
    cursor = conn.cursor()
    try:
        cursor.execute("""
            INSERT INTO articles (title, date, author, url, source)
            VALUES (?, ?, ?, ?, ?)
        """, (title, date, author, url, source))
        conn.commit()
        return True
    except sqlite3.IntegrityError:
        return False
    finally:
        conn.close()


In [6]:
def montar_url(pagina):
    if pagina == 1:
        return "https://www12.senado.leg.br/noticias/ultimas"
    return f"https://www12.senado.leg.br/noticias/ultimas/{pagina}"


In [7]:
noticias = []

for pagina in range(5, 0, -1):
    url = montar_url(pagina)
    print(f"📄 Coletando página {pagina}: {url}")

    r = requests.get(url, timeout=10)
    soup = BeautifulSoup(r.text, "html.parser")

    lista = soup.find("ol", class_="lista-resultados")
    if not lista:
        print("⚠️ Nenhuma lista encontrada")
        continue

    itens = lista.find_all("li")
    print(f"   {len(itens)} notícias encontradas")

    for item in itens:
        titulo_tag = item.find("span", class_="eta")
        link_tag = item.find("a")
        data_tag = item.select_one("div.text-muted.normalis")

        if not titulo_tag or not link_tag:
            continue

        data = {
            "title": titulo_tag.get_text(strip=True),
            "date": data_tag.get_text(strip=True) if data_tag else None,
            "author": "Agência Senado",
            "url": "https://www12.senado.leg.br" + link_tag["href"],
            "source": "Senado Federal"
        }

        noticias.append(data)
        insert_article(**data)

    time.sleep(1)

print(f"\n✅ Total coletado: {len(noticias)} notícias")
df_senado = pd.DataFrame(noticias)
display(df_senado.head())


📄 Coletando página 5: https://www12.senado.leg.br/noticias/ultimas/5
   20 notícias encontradas
📄 Coletando página 4: https://www12.senado.leg.br/noticias/ultimas/4
   20 notícias encontradas
📄 Coletando página 3: https://www12.senado.leg.br/noticias/ultimas/3
   20 notícias encontradas
📄 Coletando página 2: https://www12.senado.leg.br/noticias/ultimas/2
   20 notícias encontradas
📄 Coletando página 1: https://www12.senado.leg.br/noticias/ultimas
   20 notícias encontradas

✅ Total coletado: 97 notícias


,title,date,author,url,source
0,"Para advogados, proposta de Código Civil pode ...",11/12/2025 16h11,Agência Senado,https://www12.senado.leg.br/noticias/materias/...,Senado Federal
1,"Aprovado na Câmara, projeto do devedor contuma...",11/12/2025 15h55,Agência Senado,https://www12.senado.leg.br/noticias/materias/...,Senado Federal
2,Girão aponta conflito de interesses de Toffoli...,11/12/2025 15h16,Agência Senado,https://www12.senado.leg.br/noticias/materias/...,Senado Federal
3,Marco do Sistema de Pagamentos Brasileiro avança,11/12/2025 15h14,Agência Senado,https://www12.senado.leg.br/noticias/audios/20...,Senado Federal
4,PL da Dosimetria será analisado na CCJ na quar...,11/12/2025 14h51,Agência Senado,https://www12.senado.leg.br/noticias/materias/...,Senado Federal


In [8]:
def load_articles():
    conn = sqlite3.connect(DATABASE_NAME)
    df = pd.read_sql("""
        SELECT * FROM articles
        ORDER BY date DESC
    """, conn)
    conn.close()
    return df

df_db = load_articles()
print(f"📦 Total no banco: {len(df_db)} registros")
display(df_db.head(20))


📦 Total no banco: 609 registros


,id,title,date,author,url,source
0,74,"Em duas décadas, proporção de lares urbanos br...",31 OUT 2024,CGI.br Notícias,https://cgi.br/noticia/releases/em-duas-decada...,CGI Notícias
1,188,"No FIB13, especialistas apontam expectativas e...",31 MAI 2023,CGI.br Notícias,https://cgi.br/noticia/releases/no-fib-13-espe...,CGI Notícias
2,12,Mais acessível e completo: conheça o novo site...,30 SET 2025,CGI.br Notícias,https://cgi.br/noticia/notas/mais-acessivel-e-...,CGI Notícias
3,13,Evento em São Paulo celebra 15 anos de SACI-Ad...,30 SET 2025,CGI.br Notícias,https://cgi.br/noticia/releases/evento-em-sao-...,CGI Notícias
4,189,Abertas as inscrições para o 13º Fórum da Inte...,30 MAI 2023,CGI.br Notícias,https://cgi.br/noticia/releases/abertas-as-ins...,CGI Notícias
5,119,Declaração final do NETmundial+10 apresenta re...,30 ABR 2024,CGI.br Notícias,https://cgi.br/noticia/releases/declaracao-fin...,CGI Notícias
6,83,Nota de falecimento: Mariana Stanton,29 SET 2024,CGI.br Notícias,https://cgi.br/noticia/notas/nota-de-falecimen...,CGI Notícias
7,159,Nota de falecimento: Luiz Gonzaga Lauschner,29 SET 2023,CGI.br Notícias,https://cgi.br/noticia/notas/nota-de-falecimen...,CGI Notícias
8,38,Fórum da Internet no Brasil resgata legado da ...,29 MAI 2025,CGI.br Notícias,https://cgi.br/noticia/releases/forum-da-inter...,CGI Notícias
9,39,Prêmio Destaques homenageia 12 personalidades ...,29 MAI 2025,CGI.br Notícias,https://cgi.br/noticia/releases/premio-destaqu...,CGI Notícias


In [9]:
def plot_charts(df):
    if df.empty:
        print("❌ Sem dados para plotar")
        return

    # Top 15
    top15 = df.head(15).copy()
    top15["rank"] = range(1, len(top15) + 1)

    fig1 = px.bar(
        top15,
        x="rank",
        y="title",
        orientation="h",
        title="Top 15 Notícias – Internet Governance"
    )
    fig1.update_layout(height=600)
    fig1.show()

    # Fonte
    source_count = df["source"].value_counts().reset_index()
    source_count.columns = ["source", "count"]

    fig2 = px.pie(
        source_count,
        names="source",
        values="count",
        title="Distribuição por Fonte"
    )
    fig2.show()

    # Palavras
    text = " ".join(df["title"].astype(str)).lower()
    words = re.findall(r"\b\w{4,}\b", text)

    word_freq = (
        pd.Series(words)
        .value_counts()
        .head(20)
        .reset_index()
    )
    word_freq.columns = ["palavra", "freq"]

    fig3 = px.treemap(
        word_freq,
        path=["palavra"],
        values="freq",
        title="Palavras mais frequentes nos títulos"
    )
    fig3.show()


In [10]:
plot_charts(df_db)